保存 OpenAI API Key 為環境變量

In [1]:
from getpass import getpass
import os
from dotenv import load_dotenv

# 載入環境變數
load_dotenv()
# 兩個 API 的密鑰
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")

In [2]:
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator

# 創建系統消息和用戶消息的 ChatMessage 對象
messages = [
    ChatMessage.from_system(
        "即使某些輸入資料採用其他語言，也始終以繁體中文回應。"
    ),
    ChatMessage.from_user(
        "什麼是自然語言處理？要簡潔。"
    ),
]

# 初始化 OpenAIChatGenerator
chat_generator = OpenAIChatGenerator(model="gpt-4-turbo")
# 傳入消息並運行
chat_generator.run(messages=messages)

/Users/samhsiao/Documents/PythonVenv/envHaystack/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'replies': [ChatMessage(content='自然語言處理（NLP）是一們資訊科技領域，致力於讓計算機能夠理解、解釋、操作和生成人類語言。这项技术涉及語言學、計算機科學和人工智慧的交叉發展。', role=<ChatRole.ASSISTANT: 'assistant'>, name=None, meta={'model': 'gpt-4-turbo-2024-04-09', 'index': 0, 'finish_reason': 'stop', 'usage': {'completion_tokens': 96, 'prompt_tokens': 67, 'total_tokens': 163}})]}

In [3]:
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.generators.utils import print_streaming_chunk

# 使用流式回調函數初始化 OpenAIChatGenerator
chat_generator = OpenAIChatGenerator(
    model="gpt-4-turbo",
    streaming_callback=print_streaming_chunk
)
# 傳入消息並運行
response = chat_generator.run(messages=messages)

自然語言處理是人工普遍和語言學領域的一部分，它幫助計算機理解、解釋、操作和生成人類語言。

In [4]:
from haystack import Pipeline, Document
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.writers import DocumentWriter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

# 創建文件
documents = [
    Document(content="我的名字是 Jean，我住在 Paris。"),
    Document(content="我的名字是 Mark，我住在 Berlin。"),
    Document(content="我的名字是 Giorgio，我住在 Rome。"),
    Document(content="我的名字是 Marta，我住在 Madrid。"),
    Document(content="我的名字是 Harry，我住在 London。"),
]

# 初始化內存文件儲存
document_store = InMemoryDocumentStore()

# 創建索引管道
indexing_pipeline = Pipeline()
indexing_pipeline.add_component(
    instance=SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2"), name="doc_embedder"
)
indexing_pipeline.add_component(
    instance=DocumentWriter(document_store=document_store),
    name="doc_writer"
)

# 連接嵌入器和文件寫入器
indexing_pipeline.connect(
    "doc_embedder.documents",
    "doc_writer.documents"
)

# 運行管道
indexing_pipeline.run({
    "doc_embedder": {"documents": documents}
})

/Users/samhsiao/Documents/PythonVenv/envHaystack/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py:174: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v3 of SentenceTransformers.
  warnings.warn(
/Users/samhsiao/Documents/PythonVenv/envHaystack/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Batches: 100%|██████████| 1/1 [00:01<00:00,  2.00s/it]


{'doc_writer': {'documents_written': 5}}

In [5]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator

# 定義提示模板
template = """
根據給定的上下文回答問題。

上下文:
{% for document in documents %}
    {{ document.content }}
{% endfor %}
問題: {{ question }}
答案:
"""

# 創建 RAG 管道
rag_pipe = Pipeline()
rag_pipe.add_component(
    "embedder",
    SentenceTransformersTextEmbedder(
        model="sentence-transformers/all-MiniLM-L6-v2"
    )
)
rag_pipe.add_component(
    "retriever",
    InMemoryEmbeddingRetriever(
        document_store=document_store
    )
)
rag_pipe.add_component(
    "prompt_builder",
    PromptBuilder(
        template=template
    )
)
rag_pipe.add_component(
    "llm",
    OpenAIGenerator(model="gpt-4-turbo")
)

# 連接組件
rag_pipe.connect(
    "embedder.embedding",
    "retriever.query_embedding"
)
rag_pipe.connect(
    "retriever",
    "prompt_builder.documents"
)
rag_pipe.connect(
    "prompt_builder",
    "llm"
)

🚅 Components
  - embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [6]:
query = "Mark 住在哪裡？"
rag_pipe.run({
    "embedder": {"text": query},
    "prompt_builder": {"question": query}
})

Batches: 100%|██████████| 1/1 [00:01<00:00,  1.23s/it]


{'llm': {'replies': ['Mark 住在 Berlin。'],
  'meta': [{'model': 'gpt-4-turbo-2024-04-09',
    'index': 0,
    'finish_reason': 'stop',
    'usage': {'completion_tokens': 7,
     'prompt_tokens': 127,
     'total_tokens': 134}}]}}

In [7]:
def rag_pipeline_func(query: str):
    result = rag_pipe.run({
        "embedder": {"text": query},
        "prompt_builder": {"question": query}
    })
    return {"reply": result["llm"]["replies"][0]}

In [8]:
WEATHER_INFO = {
    "Berlin": {
        "weather": "mostly sunny", "temperature": 7, "unit": "celsius"
    },
    "Paris": {
        "weather": "mostly cloudy", "temperature": 8, "unit": "celsius"
    },
    "Rome": {
        "weather": "sunny", "temperature": 14, "unit": "celsius"
    },
    "Madrid": {
        "weather": "sunny", "temperature": 10, "unit": "celsius"
    },
    "London": {
        "weather": "cloudy", "temperature": 9, "unit": "celsius"
    },
}

def get_current_weather(location: str):
    if location in WEATHER_INFO:
        return WEATHER_INFO[location]
    else:
        # 回退數據
        return {
            "weather": "sunny",
            "temperature": 21.8,
            "unit": "fahrenheit"
        }

In [9]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "rag_pipeline_func",
            "description": "獲取有關人們居住地點的信息",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "搜尋中使用的查詢。從用戶的消息中推斷出這一點。它應該是一個問題或一個陳述。",
                    }
                },
                "required": ["query"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "取得當前天氣",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "城市和州，例如加州舊金山"
                    }
                },
                "required": ["location"],
            },
        },
    },
]

In [15]:
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.generators.utils import print_streaming_chunk

# 創建消息列表，包含系統消息和用戶查詢
messages = [
    ChatMessage.from_system(
        "不要假設將哪些值插入函數中。如果用戶要求不明確，請要求澄清。"
    ),
    ChatMessage.from_user("你能告訴我 Tom 住在哪裡嗎？"),
]

# 初始化 OpenAIChatGenerator
chat_generator = OpenAIChatGenerator(
    model="gpt-4-turbo",
    streaming_callback=print_streaming_chunk
)
# 傳入消息和工具列表並運行
response = chat_generator.run(
    messages=messages,
    generation_kwargs={"tools": tools}
)
# 輸出查看
print(response)

抱歉，我無法提供關於特定人物如 Tom 的私人住址信息，除非您能提供更多的背景資料或明確的公開資訊需求。如果有其他問題或需要幫助，請告訴我！{'replies': [ChatMessage(content='抱歉，我無法提供關於特定人物如 Tom 的私人住址信息，除非您能提供更多的背景資料或明確的公開資訊需求。如果有其他問題或需要幫助，請告訴我！', role=<ChatRole.ASSISTANT: 'assistant'>, name=None, meta={'model': 'gpt-4-turbo-2024-04-09', 'index': 0, 'finish_reason': 'stop', 'usage': {}})]}


In [16]:
import json

# 解析函數調用信息
# 提取第一個回應中的 content
content = response['replies'][0].content

# 將 content 解析為 JSON
# content 是一個 JSON 字串，需要轉換為 Python 字典
function_calls = json.loads(content)

# 提取第一個函數調用信息
# 提取函數調用列表中的第一個元素
function_call = function_calls[0]

# 獲取函數名稱
# 獲取函數名稱，這是我們需要調用的函數
function_name = function_call['function']['name']

# 解析函數參數
# 將參數解析為字典格式
function_args = json.loads(function_call['function']['arguments'])

# 打印函數名稱和參數
print("Function Name:", function_name)
print("Function Arguments:", function_args)

# 定義可用的函數
def rag_pipeline_func(query: str):
    # 這裡假設你的 `rag_pipeline_func` 函數定義
    return {"reply": f"Mark lives in Berlin, query was: {query}"}

def get_current_weather(location: str):
    # 這裡假設你的 `get_current_weather` 函數定義
    return {"weather": "sunny", "temperature": 20, "location": location}

# 可用函數字典
available_functions = {
    "rag_pipeline_func": rag_pipeline_func,
    "get_current_weather": get_current_weather
}

# 查找相應的函數並使用給定的參數調用它
if function_name in available_functions:
    # 根據函數名稱找到對應的函數
    function_to_call = available_functions[function_name]
    # 使用解包操作將參數傳遞給函數
    function_response = function_to_call(**function_args)
    # 打印函數的返回值
    print("Function Response:", function_response)
else:
    # 如果函數名稱未找到，打印錯誤訊息
    print(f"Function {function_name} not found.")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [17]:
from haystack.dataclasses import ChatMessage

# 創建函數回應消息
function_message = ChatMessage.from_function(
    content=json.dumps(function_response),
    name=function_name
)
# 將函數回應消息添加到消息列表
messages.append(function_message)

# 再次運行 OpenAIChatGenerator
response = chat_generator.run(
    messages=messages,
    generation_kwargs={"tools": tools}
)

對不起，我沒辦法提供有關 Tom 的具體居住地點。您需要詳細信息或有其他問題嗎？

In [18]:
import gradio as gr
import json

from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator

chat_generator = OpenAIChatGenerator(model="gpt-3.5-turbo")
response = None
messages = [
    ChatMessage.from_system(
        "不要假設將哪些值插入函數中。"
        "如果用戶要求不明確，請要求澄清。"
    )
]

# 定義聊天機器人函數
def chatbot_with_fc(message, history):
    messages.append(ChatMessage.from_user(message))
    response = chat_generator.run(messages=messages, generation_kwargs={"tools": tools})

    while True:
        # 如果 OpenAI 回應是一個工具調用
        if response and response["replies"][0].meta["finish_reason"] == "tool_calls":
            function_calls = json.loads(response["replies"][0].content)
            print(response["replies"][0])
            for function_call in function_calls:
                # 解析函數調用信息
                function_name = function_call["function"]["name"]
                function_args = json.loads(function_call["function"]["arguments"])

                # 查找相應的函數並使用給定的參數調用它
                function_to_call = available_functions[function_name]
                function_response = function_to_call(function_args)

                # 使用 `ChatMessage.from_function` 將函數回應添加到消息列表
                messages.append(ChatMessage.from_function(content=json.dumps(function_response), name=function_name))
                response = chat_generator.run(messages=messages, generation_kwargs={"tools": tools})

        # 常規對話
        else:
            messages.append(response["replies"][0])
            break
    return response["replies"][0].content

# 創建聊天界面
demo = gr.ChatInterface(
    fn=chatbot_with_fc,
    # 顯示在下方的範例欄位
    examples=[
        "你能告訴我 Giorgio 住在哪裡嗎？",
        "Madrid 的天氣怎麼樣？",
        "誰住在 London?",
        "Mark 住的地方的天氣怎麼樣？",
    ],
    title="請詢問有關天氣或人們居住的地方。",
)

In [19]:
# 啟動聊天應用程序
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


ChatMessage(content='[{"id": "call_iHmsDkbsiYIoVVRZMYeGE6Pb", "function": {"arguments": "{\\"query\\":\\"Tom\\u4f4f\\u54ea\\u88e1\\uff1f\\"}", "name": "rag_pipeline_func"}, "type": "function"}]', role=<ChatRole.ASSISTANT: 'assistant'>, name=None, meta={'model': 'gpt-3.5-turbo-0125', 'index': 0, 'finish_reason': 'tool_calls', 'usage': {'completion_tokens': 23, 'prompt_tokens': 217, 'total_tokens': 240}})
ChatMessage(content='[{"id": "call_PbEX28ukt89LbvIzbUsqYrvG", "function": {"arguments": "{\\"query\\":\\"Who lives in what city?\\"}", "name": "rag_pipeline_func"}, "type": "function"}]', role=<ChatRole.ASSISTANT: 'assistant'>, name=None, meta={'model': 'gpt-3.5-turbo-0125', 'index': 0, 'finish_reason': 'tool_calls', 'usage': {'completion_tokens': 21, 'prompt_tokens': 288, 'total_tokens': 309}})
ChatMessage(content='[{"id": "call_F2k4ZZxC1gHRv6bdNiRn4m4a", "function": {"arguments": "{\\"location\\":\\"\\u67cf\\u6797\\"}", "name": "get_current_weather"}, "type": "function"}]', role=<Chat